# Non-Gaussian Circuit with Strawberry Fields

In this example, we introduce PennyLane’s [Strawberry Fields plugin](https://github.com/XanaduAI/pennylane-sf) and use it to explore a non-Gaussian photonic circuit. 

<div class="admonition note">

<div class="admonition-title">Note</div>

To follow along with this tutorial on your own computer, you will require:

- The `PennyLane-SF` plugin in order to access the Strawberry Fields Fock backend using PennyLane. It can be installed via pip:

`pip install pennylane-sf`

</div>

</div>

### A Non-Gaussian Circuit

We will be considering the following photonic circuit:

Breaking this down, step-by-step:

1. **We start the computation with two qumode subsystems.** In PennyLane, we use the shorthand wires to refer to quantum subsystems, whether they are qumodes, qubits or any other kind of quantum register.

2. **Prepare the state $\mid1,0\rangle$.** Hence, wire 0 is prepared in a single-photon state while wire 1 is prepared in the vacuum state. The former state is non-Gaussian, necessitating the use of the `strawberryfields.fock` backend device.

3. **Both wires are then incident on a beamsplitter with free parameters $\theta$ and $\phi$.** Here, we have the convention that the beamsplitter transmission amplitude is $t=\cos\theta$ and the reflection amplitude is $r=e^{i\phi}\sin\theta$. 

4. **Finally, we measure the mean photon number $\langle\hat{n}\rangle$ of the second wire**, where $\hat{n}=\hat{a}^{\dagger}\hat{a}$ is the number operator acting on the Fock basis such that $\hat{n}\mid n\rangle=n \mid n\rangle$.

The **aim** of this example is to optimize the beamsplitter parameters $(\theta,\phi)$ such that the expected photon number of the second wire is maximized. Since the beamsplitter is a passive optical element that preserves the total photon number, this corresponds to the output state $\mid0,1\rangle$ — i.e., when the incident photon from the first wire has been ‘redirected’ to the second wire.

### Exact Calculation

The initial state of the circuit is $\psi_0=\mid1,0\rangle$. The output state of the system is of the form $\psi=a \mid1,0\rangle+b \mid0,1\rangle$ where $\mid a\mid^2 + \mid b\mid^2 = 1$.

We may thus write the output state as a vector in this computational basis $\psi=\begin{pmatrix} a \\ b\end{pmatrix}$.

The beamsplitter acts on this two-dimensional subspace as follows:

$$ \psi=B(\theta,\phi)\mid1,0\rangle=\begin{pmatrix} \cos\theta &  -e^{-i\phi}\sin\theta \\ e^{i\phi}\sin\theta & \cos\theta \end{pmatrix}\begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} \cos\theta \\ e^{i\phi}\sin\theta \end{pmatrix} $$

Furthermore, the mean photon number of the second wire is:

$$\langle\hat{n}_1\rangle=\langle\psi\mid\hat{n}_1\mid\psi\rangle=\mid e^{i\phi}\sin\theta\mid^2 \langle0,1\mid\hat{n}_1\mid0,1\rangle=\sin^2\theta $$

Therefore, we can see that:

1. $0\leq\hat{n}_1\leq1$ : the output of the quantum circuit is bound between 0 and 1

2. $\frac{\partial}{\partial\phi}\langle\hat{n}_1\rangle =0$: the output of the quantum circuit is independent of the beamsplitter phase  $\phi$

3. The output of the quantum circuit above is maximised when  $\theta=(2m+1)\pi/2$ for $m \in \mathbb{Z}_0$

### Loading The Plugin Device

While PennyLane provides a basic qubit simulator (`default.qubit`) and a basic CV Gaussian simulator (`default.gaussian`), the true power of PennyLane comes from its [plugin ecosystem](https://pennylane.readthedocs.io/en/latest/plugins.html), allowing quantum computations to be run on a variety of quantum simulator and hardware devices.

For this circuit, we will be using the `strawberryfields.fock` device to construct a `QNode`. This allows the underlying quantum computation to be performed using the Strawberry Fields Fock backend.

In [1]:
# lets first import the essentials
import pennylane as qml
from pennylane import numpy as np

Next, we create a device to run the quantum node. This is easy in PennyLane; as soon as the `PennyLane-SF` plugin is installed, the `strawberryfields.fock` device can be loaded — no additional commands or library imports required.

In [2]:
dev_fock = qml.device('strawberryfields.fock', wires=2, cutoff_dim=2)

Compared to the default devices provided with PennyLane, the `strawberryfields.fock` device requires the additional keyword argument:
- `cutoff_dim`: the Fock space truncation used to perform the quantum simulation

<div class="admonition important">

<div class="admonition-title">Note</div>

Devices provided by external plugins may require additional arguments and keyword arguments — consult the plugin documentation for more details.

</div>

</div>

### Constructing the QNode

Now that we have initialized the device, we can construct our quantum node. Like the other tutorials, we use the `qnode decorator` to convert our quantum function (encoded by the circuit above) into a quantum node running on Strawberry Fields.

In [3]:
@qml.qnode(dev_fock)
def photon_redirection(params):
    qml.FockState(1, wires=0)
    qml.Beamsplitter(params[0], params[1], wires=[0, 1])
    return qml.expval.MeanPhoton(1)

The `strawberryfields.fock` device supports all CV objects provided by PennyLane; see [CV operations](https://pennylane.readthedocs.io/en/latest/code/ops/cv.html#cv-ops) and [CV expectations](https://pennylane.readthedocs.io/en/latest/code/expval/cv.html#cv-expval).

### Optimization

Let’s now use one of the built-in PennyLane optimizers in order to carry out photon redirection. Since we wish to maximize the mean photon number of the second wire, we can define our cost function to *minimize* the negative of the circuit output.

In [4]:
def cost(params):
    return -photon_redirection(params)

To begin our optimization, let’s choose the following small initial values of $\theta$ and $\phi$:

In [5]:
init_params = np.array([0.01, 0.01])
cost(init_params)

-9.999666671111085e-05

Here, we choose the values of $\theta$ and $\phi$ to be very close to zero; this results in $B(\theta,\phi)\approx \hat{I}$, and the output of the quantum circuit will be very close to $\mid1,0\rangle$ — i.e., the circuit leaves the photon in the first mode.

<div class="admonition note">

<div class="admonition-title">Note</div>

Why don’t we choose $\theta=0$ and $\phi=0$ as the initial values?

At this point in the parameter space:

$$\langle\hat{n}_1\rangle=0$$

$$\frac{\partial}{\partial\theta}\langle\hat{n}_1\rangle|_{\theta=0}=2\sin\theta\cos\theta|_{\theta=0}=0$$

Since the gradient is zero at these initial parameter values, the optimization algorithm would never descend from the maximum.
</div>

</div>

This can also be verified directly using PennyLane `grad()` function:

In [6]:
dphoton_redirection = qml.grad(photon_redirection, argnum=0)
dphoton_redirection([0., 0.])

[0.0, 0.0]

Now, let’s use the `GradientDescentOptimizer` and update the circuit parameters over 100 optimization steps.

In [7]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i+1) % 5 == 0:
        print('Cost after step {:5d}: {: .7f}'.format(i+1, cost(params)) )

Cost after step     5: -0.0349558
Cost after step    10: -0.9969017
Cost after step    15: -1.0000000
Cost after step    20: -1.0000000
Cost after step    25: -1.0000000
Cost after step    30: -1.0000000
Cost after step    35: -1.0000000
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000


Try this yourself — the optimization should converge quickly, giving the following values of $\theta$ and $\phi$:

In [8]:
print('Optimized rotation angles: {}'.format(params))

Optimized rotation angles: [1.57079633 0.01      ]


Comparing this to the exact calculation above, this is close to the optimum value of $\theta=\pi/2$, while the value of $\phi$ has not changed — consistent with the fact that $\langle\hat{n}_1\rangle$ is independent of $\phi$.